In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os
proj = os.path.dirname(os.getcwd())
if proj not in sys.path:
    sys.path.insert(0, proj)
sys.path

['/Volumes/Projects/Evidently/record-thing',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python311.zip',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/lib-dynload',
 '',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages']

In [3]:
# Device independent code
import torch

device = "cpu" 
if torch.cuda.is_available():
    device = "cuda"
# if torch.backends.mps.is_available:
#     device = torch.device("mps")
torch.device(device)
torch.set_default_device(device=device)

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [4]:
!docker compose up -d

/Users/henrikvendelbo/.zshenv:.:1: no such file or directory: /Users/henrikvendelbo/.cargo/env
[+] Running 1/0
 ✔ Container rqlite_server  Running                                        0.0s 


In [6]:
from dataset.db import init_db_sqlite
connection = init_db_sqlite(disconnect=False)
connection

# UPC/QR lookup

If the scanner recognises a barcode or QR code the product number should be looked up.
If multiple matches are found the category of product should be used to pick the most likely match.
The user is asked to confirm the match YES/NO.

In [10]:
from dataset.product import lookup_product
from dataset.ingestion import Ingestor
import pandas as pd

upc = '0885909950805'
with Ingestor(connection.cursor(), device=device) as ingestor:

    result = lookup_product(upc, ingestor=ingestor, save=True)
    if result:
        print(result)
        display(pd.DataFrame([result], columns=["id", "name", "upc","asin","elid","brand","model","color","tags","category","title","description"]))

        product_images = ingestor.cursor.execute("SELECT * FROM product_images WHERE product_id = ?", [result.id])
        display(pd.DataFrame(product_images.fetchall(), columns=[desc[0] for desc in product_images.description]))
        # match with product["items"][0]["images"] product similarity with pictures of the recording.
        # print(number_matches, upc, item)

Creating product images: [ImageAssetDescription(url=http://img.bbystatic.com/BestBuy_US/images/products/7218/7218034_sc.jpg, image=<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=245x500 at 0x165B891D0>), ImageAssetDescription(url=https://i5.walmartimages.com/asr/816d5a79-828f-4bc9-9033-1f8f08213857_1.a409ced9502bcea4cf84e7ab7b60c58f.jpeg?odnHeight=450&odnWidth=450&odnBg=ffffff, image=<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=450x450 at 0x300833490>), ImageAssetDescription(url=https://c1.neweggimages.com/ProductImageCompressAll640/A9K8_131002154351685402J2X9S9guNK.jpg, image=<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x30093A8D0>), ImageAssetDescription(url=http://img1.r10.io/PIC/112231913/0/1/250/112231913.jpg, image=None)]
id=b'2px8B6H9FzygNfIJ3PNWKUtTlNm' upc='885909950805' asin='B00NQGOZV0' elid='195389220744' brand='Apple' model='Apple iPhone 6, Space Gray, 64 GB (T-Mobile)' color=',,' category=None title=None description='Electronics > C

,id,name,upc,asin,elid,brand,model,color,tags,category,title,description
0,"(id, b'2px8B6H9FzygNfIJ3PNWKUtTlNm')","(upc, 885909950805)","(asin, B00NQGOZV0)","(elid, 195389220744)","(brand, Apple)","(model, Apple iPhone 6, Space Gray, 64 GB (T-M...","(color, ,,)","(category, None)","(title, None)","(description, Electronics > Communications > T...","(name, Display Retina HD display 1334-by-750...","(tags, Apple iPhone 6, Space Gray, 64 GB (T-Mo..."


,id,product_id,embedding_id,url,sha1,cache_path,is_public,image_type,parse_date
0,2px8B6oedu0MhdJb1laIjCSJe5U,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8B5dcGln0NmvNkV7akJuD68j,http://img.bbystatic.com/BestBuy_US/images/pro...,bac74ef3b8d755e3cdda66976df04526be779218,None,1,web_scraped,2024-12-08 22:17:07
1,2px8B626j2pXUh7Lg6CXv4IUndk,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8B7AhZKDtTzOGPPkWEfwScgA,https://i5.walmartimages.com/asr/816d5a79-828f...,6a6368ca3f16e0f567603b5a37bb2582dbad92f1,None,1,web_scraped,2024-12-08 22:17:07
2,2px8B6QJ6SYcYu8JGHS8sMoRZwz,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8B5ESMnB60cTJq7WomuqJFmw,https://c1.neweggimages.com/ProductImageCompre...,e4870371708d0b8034148409208ad2ffa370abf5,None,1,web_scraped,2024-12-08 22:17:07
3,2px8cK800JF9fTbdLjkAl8AChzg,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8cGh8VBRXq2pWa2zCibuZCqz,http://img.bbystatic.com/BestBuy_US/images/pro...,bac74ef3b8d755e3cdda66976df04526be779218,None,1,web_scraped,2024-12-08 22:20:43
4,2px8cJbkWpoVhIR7l2x8dXCWw10,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8cGBUlG5XcN9U8lPQG1Vk4U2,https://i5.walmartimages.com/asr/816d5a79-828f...,6a6368ca3f16e0f567603b5a37bb2582dbad92f1,None,1,web_scraped,2024-12-08 22:20:43
5,2px8cHY75ofgiFQtqxNvu8PMmUY,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8cHyeNi1Rm2t7V2Zzrg5MY4f,https://c1.neweggimages.com/ProductImageCompre...,e4870371708d0b8034148409208ad2ffa370abf5,None,1,web_scraped,2024-12-08 22:20:43
6,2px8hYZ08cVVn0IqAQoJCnZDZdO,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8hYPa8WYCnp0O4BD7XE6RXA2,http://img.bbystatic.com/BestBuy_US/images/pro...,bac74ef3b8d755e3cdda66976df04526be779218,None,1,web_scraped,2024-12-08 22:21:25
7,2px8hUxyRGx8wMfgcZiihbOmJZq,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8hYmtLWnq7XU8eMEf9EQpDGU,https://i5.walmartimages.com/asr/816d5a79-828f...,6a6368ca3f16e0f567603b5a37bb2582dbad92f1,None,1,web_scraped,2024-12-08 22:21:25
8,2px8hYzgOX7erPxYPv9Q18i1Cc0,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8hYkKag4sXJVXSxfc91dOADw,https://c1.neweggimages.com/ProductImageCompre...,e4870371708d0b8034148409208ad2ffa370abf5,None,1,web_scraped,2024-12-08 22:21:25
9,2px8oP2yhSxpdUTSvgymc1R815u,b'2px8B6H9FzygNfIJ3PNWKUtTlNm',2px8oMoYmuPnIYjbgyxffqaN1bV,http://img.bbystatic.com/BestBuy_US/images/pro...,bac74ef3b8d755e3cdda66976df04526be779218,None,1,web_scraped,2024-12-08 22:22:19


In [ ]:
import pandas as pd

db = connection

# with connection.cursor() as cursor:
count = db.execute(
"""
  SELECT COUNT(*) FROM products
"""
        ).fetchone()

display(f"clip_assets count: {count[0]}")

rows = db.execute(
    """
      SELECT id, name, upc, asin, elid, brand, model, color, tags, category, title, description FROM products
      LIMIT 30
    """
    ).fetchall()


display(pd.DataFrame(rows, columns=["id", "name", "upc","asin","elid","brand","model","color","tags","category","title","description"]))


/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sqlmodel/main.py:638: SAWarning: This declarative base already contains a class with the same class name and module name as dataset.db.products.Products, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)
[autoreload of dataset.db.products failed: Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib

'clip_assets count: 1'

,id,name,upc,asin,elid,brand,model,color,tags,category,title,description
0,b'2puGrC5nHEndsfLjQ8GqXGrikaT',"Apple iPhone 6, Space Gray, 64 GB (T-Mobile)",885909950805,B00NQGOZV0,195389220744,Apple,MG5A2LL/A,Space Gray,",,",Electronics > Communications > Telephony > Mob...,"Apple iPhone 6, Space Gray, 64 GB (T-Mobile)",Display Retina HD display 1334-by-750-pixel ...


In [ ]:
from typing import Sequence
import torch
import requests

def generate_embedding(image: bytes) -> torch.Tensor:
    # dino.
    pass

def get_images(image_urls: Sequence[str]) -> Sequence[bytes]:
    images: Sequence[bytes] = [requests.get(url).content for url in image_urls]
    embeddings: Sequence[torch.Tensor] = [generate_embedding(image) for image in images]